# Imports

In [ ]:
import numpy as np
from tqdm.auto import tqdm
import nd2reader
import h5py
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from cytoolz import partial
import itertools as it
from pathlib import Path
from tqdm.auto import trange, tqdm
import dask
import distributed
from distributed import Client
from dask_jobqueue import SLURMCluster
from IPython.display import Video

In [ ]:
from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

In [ ]:
from paulssonlab.image_analysis import workflow, mosaic

# Config

In [ ]:
nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/220704/220704rbs_library_fish.nd2"
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/220718/RBS_DEG_library_20x.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(nd2_filename)

In [ ]:
# default_channels = ("YFP-DUAL", "RFP-Penta")
default_channels = ("YFP-DUAL", "RFP-PENTA")
channel_to_color = {
    "BF": "#ffffff",
    "RFP-PENTA": "#e22400",
    "RFP-Penta": "#e22400",
    "YFP-DUAL": "#f5eb00",
    # "GFP": "#76ba40",
    "Cy5": "#e292fe",
    # "Cy7": "#FF0000"
    # "BFP": "#3a87fd",
}

In [ ]:
font = ImageFont.truetype("fira/FiraSans-Medium.ttf")

In [ ]:
dask.config.set({"distributed.scheduler.allowed-failures": 10})

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="4GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster.scale(20)

In [ ]:
cluster.adapt(maximum=300)

In [ ]:
cluster

# Mosaic

In [ ]:
%%time
extrema = mosaic.get_intensity_extrema(nd2, ("YFP-DUAL", "RFP-PENTA"))

In [ ]:
extrema

In [ ]:
scaling_funcs = mosaic.get_scaling_funcs(extrema)
# scaling_funcs = get_scaling_funcs({"YFP-DUAL": (262, 8000), "RFP-Penta": (278, 8000)})

In [ ]:
o = mosaic.square_overlay(a[5], 0, 1, font=font)
plt.figure(figsize=(20, 20))
plt.imshow(o)

In [ ]:
%%time
num_t = nd2.sizes["t"]
scale = [80,40,20,10,5,1]#it.repeat(80)
#scale = np.geomspace(80, 0.4, 100)
# scale = it.repeat(0.1)
# scale = it.repeat(0.4)
# scale = np.geomspace(20, 0.05, num_t*3)# + np.linspace(0, 0, 0) #it.repeat(0.3)  # [0.3, 0.3, 0.3, 0.3]
# timepoints = range(0, 119, 30)
# timepoints = range(0, 110, 10)#[20,40,60]
timepoints = it.repeat(20)#[20]  # [20,40,60,80]
# timepoints = range(num_t)
# timepoints = it.chain(range(num_t), range(num_t), range(num_t)) #[20]#[20, 40, 60]
offset = np.array([604, 354])
animation_delayed = mosaic.mosaic_animate_scale(
    nd2_filename,
    scale,
    timepoints=timepoints,
    scaling_funcs=scaling_funcs,
    offset=offset,
    channels=default_channels,
    channel_to_color=channel_to_color,
    #overlay_func=partial(square_overlay, unit_scale=80, unit_width=0.5, font=font),
    delayed=True,
)

In [ ]:
#%%pyinstrument
a = dask.compute(animation_delayed, scheduler="sync")[0]

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(a[9])

In [ ]:
img = Image.new("RGBA", (1024, 1024), (0, 0, 0, 255))

fnt = ImageFont.truetype("fira/FiraSans-Medium.ttf", 80)
d = ImageDraw.Draw(img)

d.text((120, 900), "1 cell", font=fnt, fill=(255, 255, 255, 0))
d.rectangle([(100, 100), (1000, 1000)], outline=(255, 255, 255, 0), width=5)
plt.imshow(np.asarray(img))

In [ ]:
img = Image.fromarray(a[0], mode="F")
draw = ImageDraw.Draw(img)
draw.line((0, 0) + im.size, fill=128)

In [ ]:
%%time
# animation_future = client.compute(animation_delayed)
animation_future = [client.compute(a) for a in tqdm(animation_delayed)]

In [ ]:
%%time
a = client.gather(animation_future)

In [ ]:
client.retry([a for a in animation_future if a.status == "error"])

In [ ]:
client.gather([a for a in animation_future if a.status == "error"])

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(a[0])

In [ ]:
%%time
mosaic.export_video(
    a, "/home/jqs1/scratch/jqs1/microscopy/220704/mosaics/timelapse_zoomtest_1.mp4", fps=10
)

In [ ]:
Video(
    "/home/jqs1/scratch/jqs1/microscopy/220704/mosaics/timelapse_3x3_5fps.mp4",
    embed=True,
)

# FISH

In [ ]:
fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/220718/FISH/real_run")

In [ ]:
def get_fish_frame(filename, v, channel, t):
    with h5py.File(filename / f"fov={v}_config={channel}_t={t}") as f:
        frame = f["data"][()]
    return frame

In [ ]:
fish_colors = {
    "BF": "#ffffff",
    "RFP": "#e22400",
    # "YFP-DUAL": "#f5eb00",
    # "GFP": "#76ba40",
    "Cy5": "#e292fe",
    "Cy7": "#00faff"
    # "BFP": "#3a87fd",
}

In [ ]:
fish_scaling_funcs = get_scaling_funcs(
    {
        "BF": (40_000, 65_500),
        "RFP": (8000, 15000),
        "Cy5": (5_000, 40000),
        "Cy7": (2500, 4_000),
    }
)

In [ ]:
fish_scaling_funcs["Cy7"](2172)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["Cy5"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["RFP"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["RFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["BF", "RFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["BF", "RFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
%%time
frames = [
    colorized_frame(
        fish_colors,
        get_fish_frame,
        fish_dir,
        t,
        8,
        ["Cy7"],
        scaling_funcs=fish_scaling_funcs,
    )
    for t in trange(1, 11)
]

In [ ]:
%%time
rescaled_frames = [
    skimage.transform.rescale(f, 0.5, anti_aliasing=True, channel_axis=-1)
    for f in tqdm(frames)
]

In [ ]:
%%time
export_video(rescaled_frames, "/home/jqs1/_temp/FISH_Cy7.mp4", fps=5)

In [ ]:
!du -hs /home/jqs1/_temp/FISH_BF.mp4